In [20]:
import requests
from bs4 import BeautifulSoup

In [52]:
def getCaseInformation(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find_all("div", class_="entry-content")
    
    
    input = ""
    instruction = ""
    for element in results:
        try:
            
            title_element = element.find("h3")
            input = f"{title_element.text}\n "
            case_info_element = element.find("em")
            input = f"{input} \n {case_info_element.text}"
            text_elemets = element.find_all("p")
            for text_element in text_elemets:
                if(not (text_element.text.__contains__("Answers on next page") or text_element.text.__contains__("WordPress") or text_element.text.__contains__("eClinpath helped 1.2 million visitors") or  text_element.text.__contains__("Your feedback has been received"))):
                    input = f"{input} \n{text_element.text}"
        except:
            input = ""
        
        try:    
            question_elements = element.find_all("li") 
            for question_element in question_elements:
                instruction = f"{question_element.text}\n {instruction}"
        except:
            instruction = f"Evaluate these results in conjunction with the history and answer the following questions:\n {instruction}"
        
    return instruction, input

In [29]:
c


Evaluate these results in conjunction with the history and answer the following questions:
 What are potential findings in a urinalysis from this patient as a result of your top differential diagnosis?
 What is your top differential diagnosis?
 Interpret the leukogram. With no blood smear available to review, what do you think about the accuracy of the CBC differential count when looking at the WBC dot plot?
 


In [54]:
def getCaseExplanation(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find_all("div", class_="entry-content")
    
    
    output = ""
    try:
        for element in results:
            text_elemets = element.find_all("p")
            for text_element in text_elemets:
                if(not (text_element.text.__contains__("Answers on next page") or text_element.text.__contains__("WordPress") or text_element.text.__contains__("eClinpath helped 1.2 million visitors") or  text_element.text.__contains__("Your feedback has been received"))):
                    output = f"{output} \n{text_element.text}"
    except:
        output = ""
     
    return output

In [34]:
url = "https://eclinpath.com/2023-case-4/2/"
output = getCaseExplanation(url)
print(output)

 
Marked suppurative inflammation (exudative effusion) with barium crystals 
Direct smears, prepared from the submitted peritoneal fluid, were moderately to highly cellular, consisting of inflammatory cells in a small amount of blood. A differential leukocyte count was 96% non-degenerate neutrophils and 4% macrophages. There were many variably sized, irregularly shaped, pale yellow to colorless, refractile material (barium, presumptive) in the background, as well as phagocytized within neutrophils and macrophages (Question 2) (Figure 1a-2a). No infectious organisms or neoplastic cells were observed within the smears. Given the numeric data and the cytologic findings, the effusion was classified as an exudate (Question 1). 
Figure 1a: Peritoneal fluid direct smear (50x objective) 
Figure 2a: Peritoneal fluid direct smear (50x objective) 
  
The perforated gastric ulcer and suppurative peritonitis explained the inflammatory leukogram, which had likely been there for several days, given t

In [50]:
def getCaseLinks(url):
    

    caseLinks = []
    for i in range(1,13):
        print(i)
        if i == 1:
            url_page = url
        else:
            url_page = f"{url}page/{i}/"
        
        page = requests.get(url_page)
        soup = BeautifulSoup(page.content, "html.parser")
        titles = soup.find_all("h2", class_="entry-title ast-blog-single-element")
        for title in titles:
            links = title.find_all("a", href=True)
            for link in links:
                if link.text != "Diagnostic challenge":
                    caseLinks.append(link['href'])
    
    return caseLinks

In [56]:
import json

url_caselinks = "https://eclinpath.com/case-of-the-month/"
caseLinks = getCaseLinks(url_caselinks)
print(f"Found {len(caseLinks)} Cases to scrape")
data = []

for url in caseLinks:
    print(url)
    instruction, input = getCaseInformation(url)
    url_explanation = f"{url}/2/"
    output = getCaseExplanation(url_explanation)
    data.append({
        "instruction": instruction,
        "input": input,
        "output": output
    })
    
    
with open("scrapped_cases_eclinpath.json", "w") as json_file:
    json.dump(data, json_file, indent=4)

1
2
3
4
5
6
7
8
9
10
11
12
Found 114 Cases to scrape
https://eclinpath.com/case-1-2024/
https://eclinpath.com/2023-case-5/
https://eclinpath.com/2023-case-4/
https://eclinpath.com/2023-case-3/
https://eclinpath.com/2023-case-2/
https://eclinpath.com/2023-case-1/
https://eclinpath.com/2022-case-6/
https://eclinpath.com/2022-case-5/
https://eclinpath.com/2022-case-4/
https://eclinpath.com/2022-case-3/
https://eclinpath.com/2022-case-2/
https://eclinpath.com/2022-case-1/
https://eclinpath.com/2021-case-6/
https://eclinpath.com/2021-case-5/
https://eclinpath.com/2021-case-4/
https://eclinpath.com/2021-case-3/
https://eclinpath.com/2021-case-2/
https://eclinpath.com/2021-case-1/
https://eclinpath.com/2020-case-6/
https://eclinpath.com/case-5-2020/
https://eclinpath.com/2020-case-4/
https://eclinpath.com/2020-case-__/
https://eclinpath.com/2020-case-2/
https://eclinpath.com/2020-case-1-2/
https://eclinpath.com/2019-case-9/
https://eclinpath.com/2019-case-8-2/
https://eclinpath.com/2019-case-

In [ ]:
import json

json_file= "scrapped_cases_eclinpath.json"
text = ""


with open(json_file, 'r') as file:
    data = json.load(file)

i = 0


for item in data:
    i = i+1
    text = f"{text} #### CASE {i}: Start ##### \n\n"
    text = f"{text} {item['input']}\n\n"
    text = f"{text} #### Questions ##### \n\n"
    text = f"{text} {item['instruction']}\n\n"
    text = f"{text} #### Explanation ##### \n\n"
    text = f"{text} {item['output']}\n\n"
    text = f"{text} #### CASE {i}: End ##### \n\n\n\n"

with open('rag_text.txt', 'w', newline='') as file:
    file.write(text)      
    file.close()        
